# Bedrock with LangChain using a Prompt that includes Context

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Setup

In [1]:
import json
import os
import sys
import boto3

# Add any necessary paths for custom modules
module_path = ".."
sys.path.append(os.path.abspath(module_path))

from labutils import bedrock, print_ww

# Uncomment and edit the following lines for your AWS setup
# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g., "us-west-2"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [2]:
%pip install langchain-community==0.2.11
%pip install pydantic==1.10.17
%pip install sagemaker-jupyterlab-extension-common
%pip install langchain-aws==0.1.15  # or the compatible version
%pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.5
    Uninstalling langchain-0.2.5:
      Successfully uninstalled langchain-0.2.5
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.3 MB/s eta 0:00:0000:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-aws
    Found existing installation: langchain-aws 0.1.17
    Uninstalling langchain-aws-0.1.17:
      Successfull

## Reference the Bedrock LLM

In [3]:
from langchain_aws import BedrockLLM

inference_modifier = {
    'max_tokens_to_sample': 4096, 
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"]
}

textgen_llm = BedrockLLM(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock, 
    model_kwargs=inference_modifier 
)


## Create a LangChain custom prompt template

In [4]:
from langchain.prompts import PromptTemplate

# Create a prompt template with multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""
    Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
    <customer_feedback>
    {feedbackFromCustomer}
    </customer_feedback>

    Assistant:"""
)

# Fill in the template with specific values
prompt = multi_var_prompt.format(
    customerServiceManager="Swami", 
    customerName="John Doe", 
    feedbackFromCustomer="""Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere."""
)


### Print the number of tokens in the prompt (for interest/debug)

In [5]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 125 tokens


In [6]:
print(prompt)


    Human: Create an apology email from the Service Manager Swami to John Doe in response to the following feedback that was received from the customer: 
    <customer_feedback>
    Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere.
    </customer_feedback>

    Assistant:


## Invoke the model and print the response

In [7]:
response = textgen_llm.invoke(prompt)

email = response[response.index('\n')+1:]
print_ww(email)



I would like to sincerely apologize for the poor customer service experience you recently had with
our company. Receiving your feedback was disappointing, but I appreciate you taking the time to
share your concerns.

Our goal is to always provide prompt, knowledgeable and helpful support. I am very sorry that we
fell short in responding quickly to your issue. A three day delay in getting back to you is
unacceptable. You deserve much better from us. Additionally, I apologize that the initial
troubleshooting steps we provided were incorrect. I know how frustrating it is when solutions are
not identified right away. You should expect competent assistance from our team.

I have shared your feedback with our support leadership team so we can put measures in place to
prevent this from happening again. We will be re-training our staff on prompt callback procedures
and effective troubleshooting. I assure you that we are committed to improving so that we can meet
and exceed your expectations g